In [28]:
import pandas as pd
import numpy as np
from scapy.all import PcapReader, IP, TCP, UDP, ICMP, DNS
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import (silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score)
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
import os
from datetime import datetime
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')



In [29]:

class BiLSTMSequenceFeatureExtractor:
    """Extract BiLSTM sequence features from PCAP files"""
    
    def __init__(self, sequence_length=50):
        self.sequence_length = sequence_length
        self.scaler = StandardScaler()
        
    def extract_sequence_features(self, packet):
        """Extract 13 features from a single packet (same as BiLSTM)"""
        features = []
        
        try:
            if IP in packet:
                # Packet length (normalized to 1500 max)
                features.append(min(len(packet) / 1500.0, 1.0))
                
                # IP TTL (normalized to 255 max)
                features.append(packet[IP].ttl / 255.0)
                
                # IP flags
                features.append(int(packet[IP].flags) / 7.0)
                
                # IP ID variation
                features.append((packet[IP].id % 1000) / 1000.0)
                
                if TCP in packet:
                    # TCP ports
                    features.append(packet[TCP].sport / 65535.0)
                    features.append(packet[TCP].dport / 65535.0)
                    
                    # TCP flags
                    features.append(int(packet[TCP].flags) / 63.0)
                    
                    # TCP window size
                    features.append(packet[TCP].window / 65535.0)
                    
                    # Payload length
                    payload_len = len(packet[TCP].payload)
                    features.append(min(payload_len / 1500.0, 1.0))
                    
                    # TCP options count
                    features.append(min(float(len(packet[TCP].options)) / 10.0, 1.0))
                    
                    # Payload entropy
                    features.append(self._payload_entropy(bytes(packet[TCP].payload)))
                    
                    features.extend([0.0, 0.0])
                    
                elif UDP in packet:
                    features.append(packet[UDP].sport / 65535.0)
                    features.append(packet[UDP].dport / 65535.0)
                    
                    payload_len = len(packet[UDP].payload)
                    features.append(min(payload_len / 1500.0, 1.0))
                    
                    features.extend([0.0, 0.0, 0.0, 0.0, 0.0])
                    
                else:
                    features.extend([0.0] * 10)
                
                # ICMP
                features.append(1.0 if ICMP in packet else 0.0)
                
                # DNS
                features.append(1.0 if DNS in packet else 0.0)
            else:
                features = [0.0] * 13
        
        except:
            features = [0.0] * 13
        
        return np.array(features[:13])
    
    def _payload_entropy(self, payload):
        """Calculate normalized Shannon entropy"""
        if not payload or len(payload) == 0:
            return 0.0
        
        byte_counts = np.bincount(list(payload), minlength=256)
        probabilities = byte_counts[byte_counts > 0] / len(payload)
        ent = entropy(probabilities, base=2) / 8.0
        return min(ent, 1.0)
    
    def extract_flows_from_pcap(self, pcap_file, label, max_flows=1000):
        """Extract packet flows from PCAP file"""
        print(f"\nProcessing {pcap_file} (Label: {'Steganography' if label == 1 else 'Benign'})...")
        
        flow_dict = defaultdict(list)
        
        try:
            with PcapReader(pcap_file) as pcap:
                for packet in tqdm(pcap, desc="   Reading packets"):
                    if IP not in packet:
                        continue
                    
                    # Flow identifier
                    src_ip = packet[IP].src
                    dst_ip = packet[IP].dst
                    proto = packet[IP].proto
                    
                    if TCP in packet:
                        src_port = packet[TCP].sport
                        dst_port = packet[TCP].dport
                    elif UDP in packet:
                        src_port = packet[UDP].sport
                        dst_port = packet[UDP].dport
                    else:
                        src_port = 0
                        dst_port = 0
                    
                    # Create flow key (bidirectional)
                    flow_key = tuple(sorted([
                        (src_ip, src_port, proto),
                        (dst_ip, dst_port, proto)
                    ]))
                    
                    # Extract features
                    features = self.extract_sequence_features(packet)
                    flow_dict[flow_key].append(features)
            
            print(f"   ✓ Extracted {len(flow_dict)} flows")
            return flow_dict, label
        
        except Exception as e:
            print(f"   ❌ Error: {e}")
            return {}, label
    
    def aggregate_flow_features(self, flow_dict):
        """Aggregate sequence features into single flow feature vector"""
        sequences_data = []
        
        for flow_key, packets in tqdm(flow_dict.items(), desc="   Aggregating features"):
            if len(packets) >= 3:  # Minimum 3 packets
                # Pad or truncate to sequence_length
                if len(packets) < self.sequence_length:
                    packets = packets + [np.zeros(13)] * (self.sequence_length - len(packets))
                else:
                    packets = packets[:self.sequence_length]
                
                packets_array = np.array(packets)
                
                # Calculate aggregated statistics for each feature dimension
                agg_features = []
                
                for feat_idx in range(13):
                    feat_values = packets_array[:, feat_idx]
                    
                    # Statistical features: mean, std, min, max, median
                    agg_features.append(np.mean(feat_values))
                    agg_features.append(np.std(feat_values))
                    agg_features.append(np.min(feat_values))
                    agg_features.append(np.max(feat_values))
                    agg_features.append(np.median(feat_values))
                
                sequences_data.append(np.array(agg_features))
        
        return np.array(sequences_data)  # Shape: (n_flows, 13*5=65 features)




In [ ]:
class UnsupervisedClusteringAnalysis:
    """Unsupervised clustering using BiLSTM features"""
    
    def __init__(self):
        self.X = None
        self.y_true = None
        self.scaler = StandardScaler()
        self.clusters = None
        self.kmeans = None
        self.pca = None
        self.tsne = None
        
    def load_features_from_pcaps(self, benign_pcap, stego_pcap,max_flows_per_class=500, sequence_length=50):
        
        extractor = BiLSTMSequenceFeatureExtractor(sequence_length=sequence_length)
        
        all_features = []
        all_labels = []
        
        # Process benign PCAP
        benign_flows, benign_label = extractor.extract_flows_from_pcap(benign_pcap, label=0, max_flows=max_flows_per_class)
        
        if benign_flows:
            print(f"   Aggregating benign flow features...")
            benign_features = extractor.aggregate_flow_features(benign_flows)
            all_features.append(benign_features)
            all_labels.extend([benign_label] * len(benign_features))
        
        # Process stego PCAP
        stego_flows, stego_label = extractor.extract_flows_from_pcap(
            stego_pcap, label=1, max_flows=max_flows_per_class
        )
        
        if stego_flows:
            print(f"   Aggregating stego flow features...")
            stego_features = extractor.aggregate_flow_features(stego_flows)
            all_features.append(stego_features)
            all_labels.extend([stego_label] * len(stego_features))
        
        # Combine datasets
        self.X = np.vstack(all_features)
        self.y_true = np.array(all_labels)
        
        print(f"\nCombined dataset shape: {self.X.shape}")
        print(f"  - Samples: {len(self.X):,}")
        print(f"  - Features: {self.X.shape[1]} (13 dims × 5 stats)")
        
        # Class distribution
        print(f"\nClass Distribution:")
        unique, counts = np.unique(self.y_true, return_counts=True)
        for label, count in zip(unique, counts):
            label_name = "Steganography" if label == 1 else "Benign"
            percentage = (count / len(self.y_true)) * 100
            print(f"   {label_name:<20} {count:>6,} ({percentage:>5.1f}%)")
        
        # Normalize
        print(f"\nNormalizing features...")
        self.X = self.scaler.fit_transform(self.X)
        print(f"Features normalized")
    
    def determine_optimal_k(self, k_range=range(2, 11)):
        
        inertias = []
        silhouette_scores = []
        davies_bouldin_scores = []
        calinski_harabasz_scores = []
        
        print(f"\nTesting K values from {k_range.start} to {k_range.stop-1}...")
        
        for k in tqdm(k_range, desc="   Evaluating"):
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            labels = kmeans.fit_predict(self.X)
            
            inertias.append(kmeans.inertia_)
            silhouette_scores.append(silhouette_score(self.X, labels))
            davies_bouldin_scores.append(davies_bouldin_score(self.X, labels))
            calinski_harabasz_scores.append(calinski_harabasz_score(self.X, labels))
        
        optimal_k = k_range[np.argmax(silhouette_scores)]
        
        print(f"\nResults:")
        print(f"  - Optimal K (Silhouette): {optimal_k}")
        print(f"  - Best Silhouette Score: {max(silhouette_scores):.4f}")
        
        self._plot_elbow_analysis(k_range, inertias, silhouette_scores, davies_bouldin_scores, calinski_harabasz_scores, optimal_k)
        
        return optimal_k
    
    def _plot_elbow_analysis(self, k_range, inertias, silhouette_scores, davies_bouldin_scores, calinski_harabasz_scores, optimal_k):
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        k_values = list(k_range)
        
        # Inertia
        axes[0, 0].plot(k_values, inertias, 'bo-', linewidth=2, markersize=8)
        axes[0, 0].axvline(optimal_k, color='red', linestyle='--', linewidth=2, label=f'Optimal K={optimal_k}')
        axes[0, 0].set_xlabel('K', fontsize=11, fontweight='bold')
        axes[0, 0].set_ylabel('Inertia', fontsize=11, fontweight='bold')
        axes[0, 0].set_title('Elbow Method', fontsize=12, fontweight='bold')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].legend()
        
        # Silhouette
        axes[0, 1].plot(k_values, silhouette_scores, 'go-', linewidth=2, markersize=8)
        axes[0, 1].axvline(optimal_k, color='red', linestyle='--', linewidth=2, label=f'Optimal K={optimal_k}')
        axes[0, 1].set_xlabel('K', fontsize=11, fontweight='bold')
        axes[0, 1].set_ylabel('Silhouette Score', fontsize=11, fontweight='bold')
        axes[0, 1].set_title('Silhouette Analysis', fontsize=12, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].legend()
        
        # Davies-Bouldin
        axes[1, 0].plot(k_values, davies_bouldin_scores, 'mo-', linewidth=2, markersize=8)
        axes[1, 0].axvline(optimal_k, color='red', linestyle='--', linewidth=2, label=f'Optimal K={optimal_k}')
        axes[1, 0].set_xlabel('K', fontsize=11, fontweight='bold')
        axes[1, 0].set_ylabel('Davies-Bouldin Index', fontsize=11, fontweight='bold')
        axes[1, 0].set_title('Davies-Bouldin (lower is better)', fontsize=12, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3)
        axes[1, 0].legend()
        
        # Calinski-Harabasz
        axes[1, 1].plot(k_values, calinski_harabasz_scores, 'co-', linewidth=2, markersize=8)
        axes[1, 1].axvline(optimal_k, color='red', linestyle='--', linewidth=2, label=f'Optimal K={optimal_k}')
        axes[1, 1].set_xlabel('K', fontsize=11, fontweight='bold')
        axes[1, 1].set_ylabel('Calinski-Harabasz Index', fontsize=11, fontweight='bold')
        axes[1, 1].set_title('Calinski-Harabasz (higher is better)', fontsize=12, fontweight='bold')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].legend()
        
        plt.tight_layout()
        
        if not os.path.exists('results'):
            os.makedirs('results')
        
        plt.savefig('results/01_elbow_analysis.png', dpi=300, bbox_inches='tight')
        print(f"\n Saved: results/01_elbow_analysis.png")
        plt.close()
    
    def perform_clustering(self, optimal_k):
        
        print(f"\nClustering with K={optimal_k}...")
        self.kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
        self.clusters = self.kmeans.fit_predict(self.X)
        
        print(f"Clustering completed")
        
        # Cluster statistics
        print(f"\nCluster Distribution:")
        unique, counts = np.unique(self.clusters, return_counts=True)
        for cluster_id, count in zip(unique, counts):
            percentage = (count / len(self.clusters)) * 100
            print(f"   Cluster {cluster_id}: {count:>6,} ({percentage:>5.1f}%)")
        
        # Cluster purity
        print(f"\nCluster Purity Analysis:")
        for cluster_id in np.unique(self.clusters):
            mask = self.clusters == cluster_id
            cluster_labels = self.y_true[mask]
            
            if len(cluster_labels) > 0:
                stego_count = np.sum(cluster_labels == 1)
                benign_count = np.sum(cluster_labels == 0)
                stego_pct = (stego_count / len(cluster_labels)) * 100
                
                purity = "PURE STEGO" if stego_pct > 90 else ("PURE BENIGN" if stego_pct < 10 else "MIXED")
                print(f"   Cluster {cluster_id}: {stego_pct:>5.1f}% Stego, {100-stego_pct:>5.1f}% Benign [{purity}]")
    
    def reduce_dimensions_pca(self):
        
        print(f"\nApplying PCA...")
        self.pca = PCA(n_components=2, random_state=42)
        X_pca = self.pca.fit_transform(self.X)
        
        explained = self.pca.explained_variance_ratio_.sum() * 100
        print(f"✓ Explained variance: {explained:.2f}%")
        print(f"  - PC1: {self.pca.explained_variance_ratio_[0]*100:.2f}%")
        print(f"  - PC2: {self.pca.explained_variance_ratio_[1]*100:.2f}%")
        
        return X_pca
    
    def reduce_dimensions_tsne(self):
        print("\n" + "="*70)
        print(" t-SNE DIMENSIONALITY REDUCTION")
        print("="*70)
        
        print(f"\nApplying t-SNE (may take a moment)...")
        self.tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000, verbose=1)
        X_tsne = self.tsne.fit_transform(self.X)
        
        print(f"t-SNE completed")
        return X_tsne
    
    def plot_visualizations(self, X_pca, X_tsne, optimal_k):
        """Plot all visualizations"""
        print("\n" + "="*70)
        print(" GENERATING VISUALIZATIONS")
        print("="*70)
        
        if not os.path.exists('results'):
            os.makedirs('results')
        
        # PCA visualization
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=self.clusters, cmap='tab10',alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
        axes[0].set_xlabel('PC1', fontsize=11, fontweight='bold')
        axes[0].set_ylabel('PC2', fontsize=11, fontweight='bold')
        axes[0].set_title(f'PCA - Colored by Clusters (K={optimal_k})', fontsize=12, fontweight='bold')
        axes[0].grid(True, alpha=0.2)
        
        colors = ['#2ecc71' if label == 0 else '#e74c3c' for label in self.y_true]
        axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=colors, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
        axes[1].set_xlabel('PC1', fontsize=11, fontweight='bold')
        axes[1].set_ylabel('PC2', fontsize=11, fontweight='bold')
        axes[1].set_title('PCA - Colored by True Labels', fontsize=12, fontweight='bold')
        axes[1].grid(True, alpha=0.2)
        
        from matplotlib.patches import Patch
        legend = [Patch(facecolor='#2ecc71', label='Benign'), 
                 Patch(facecolor='#e74c3c', label='Stego')]
        axes[1].legend(handles=legend)
        
        plt.tight_layout()
        plt.savefig('results/02_pca_clusters.png', dpi=300, bbox_inches='tight')
        print(f"\n✓ Saved: results/02_pca_clusters.png")
        plt.close()
        
        # t-SNE visualization
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        axes[0].scatter(X_tsne[:, 0], X_tsne[:, 1], c=self.clusters, cmap='tab10',alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
        axes[0].set_xlabel('t-SNE 1', fontsize=11, fontweight='bold')
        axes[0].set_ylabel('t-SNE 2', fontsize=11, fontweight='bold')
        axes[0].set_title(f't-SNE - Colored by Clusters (K={optimal_k})', fontsize=12, fontweight='bold')
        axes[0].grid(True, alpha=0.2)
        
        axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=colors, alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
        axes[1].set_xlabel('t-SNE 1', fontsize=11, fontweight='bold')
        axes[1].set_ylabel('t-SNE 2', fontsize=11, fontweight='bold')
        axes[1].set_title('t-SNE - Colored by True Labels', fontsize=12, fontweight='bold')
        axes[1].grid(True, alpha=0.2)
        axes[1].legend(handles=legend)
        
        plt.tight_layout()
        plt.savefig('results/02_tsne_clusters.png', dpi=300, bbox_inches='tight')
        print(f" Saved: results/02_tsne_clusters.png")
        plt.close()
        
        # Silhouette analysis
        silhouette_vals = silhouette_samples(self.X, self.clusters)
        silhouette_avg = silhouette_score(self.X, self.clusters)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        y_lower = 10
        colors_sil = plt.cm.tab10(np.linspace(0, 1, optimal_k))
        
        for i in range(optimal_k):
            cluster_silhouette_vals = silhouette_vals[self.clusters == i]
            cluster_silhouette_vals.sort()
            
            size_cluster_i = cluster_silhouette_vals.shape[0]
            y_upper = y_lower + size_cluster_i
            
            ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_vals,
                            facecolor=colors_sil[i], edgecolor=colors_sil[i], alpha=0.7)
            
            y_lower = y_upper + 10
        
        ax.axvline(x=silhouette_avg, color="red", linestyle="--", linewidth=2,label=f'Average ({silhouette_avg:.4f})')
        ax.set_xlabel("Silhouette Coefficient", fontsize=11, fontweight='bold')
        ax.set_ylabel("Cluster", fontsize=11, fontweight='bold')
        ax.set_title(f'Silhouette Plot (K={optimal_k})', fontsize=12, fontweight='bold')
        ax.set_ylim([0, len(self.X) + (optimal_k + 1) * 10])
        ax.legend()
        ax.grid(True, alpha=0.2, axis='x')
        
        plt.tight_layout()
        plt.savefig('results/03_silhouette_analysis.png', dpi=300, bbox_inches='tight')
        print(f" Saved: results/03_silhouette_analysis.png")
        plt.close()




In [31]:
def main():
    """Main pipeline"""
    benign_pcap = "Dataset/Benign_Dump.pcap"
    stego_pcap = "Dataset/steganography_dataset_20251016_233034.pcap"
    
    if not os.path.exists(benign_pcap) or not os.path.exists(stego_pcap):
        print("PCAP files not found!")
        return
    
    # Initialize
    clustering = UnsupervisedClusteringAnalysis()
    
    # Load features from PCAPs
    clustering.load_features_from_pcaps(benign_pcap, stego_pcap,max_flows_per_class=500, sequence_length=50)
    
    # Find optimal K
    optimal_k = clustering.determine_optimal_k(k_range=range(2, 11))
    
    # Perform clustering
    clustering.perform_clustering(optimal_k)
    
    # Dimensionality reduction
    X_pca = clustering.reduce_dimensions_pca()
    X_tsne = clustering.reduce_dimensions_tsne()
    
    # Plot visualizations
    clustering.plot_visualizations(X_pca, X_tsne, optimal_k)
    
    print("\nResults saved in ./results/")
    print("   - 01_elbow_analysis.png")
    print("   - 02_pca_clusters.png")
    print("   - 02_tsne_clusters.png")
    print("   - 03_silhouette_analysis.png")
    

if __name__ == "__main__":
    main()


Processing Dataset/Benign_Dump.pcap (Label: Benign)...


   Reading packets: 86000it [00:44, 1942.94it/s]


   ✓ Extracted 38444 flows
   Aggregating benign flow features...


   Aggregating features: 100%|██████████| 38444/38444 [00:00<00:00, 43572.10it/s]



Processing Dataset/steganography_dataset_20251016_233034.pcap (Label: Steganography)...


...

   ✓ Extracted 81985 flows
   Aggregating stego flow features...


   Aggregating features: 100%|██████████| 81985/81985 [00:01<00:00, 52973.93it/s] 



Combined dataset shape: (1673, 65)
  - Samples: 1,673
  - Features: 65 (13 dims × 5 stats)

Class Distribution:
   Benign                  611 ( 36.5%)
   Steganography         1,062 ( 63.5%)

Normalizing features...
Features normalized

Testing K values from 2 to 10...


   Evaluating: 100%|██████████| 9/9 [00:01<00:00,  5.72it/s]



Results:
  - Optimal K (Silhouette): 2
  - Best Silhouette Score: 0.6887

 Saved: results/01_elbow_analysis.png

Clustering with K=2...
Clustering completed

Cluster Distribution:
   Cluster 0:  1,492 ( 89.2%)
   Cluster 1:    181 ( 10.8%)

Cluster Purity Analysis:
   Cluster 0:  71.2% Stego,  28.8% Benign [MIXED]
   Cluster 1:   0.0% Stego, 100.0% Benign [PURE BENIGN]

Applying PCA...
✓ Explained variance: 48.04%
  - PC1: 35.03%
  - PC2: 13.01%

 t-SNE DIMENSIONALITY REDUCTION

Applying t-SNE (may take a moment)...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1673 samples in 0.001s...
[t-SNE] Computed neighbors for 1673 samples in 0.093s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1673
[t-SNE] Computed conditional probabilities for sample 1673 / 1673
[t-SNE] Mean sigma: 0.063075
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.832058
[t-SNE] KL divergence after 1000 iterations: 0.439305
t-SNE completed

 GENERATING VISUALIZATIONS